## Get Table Columns Description
This notebook gets the description of the tables' columns of SynPUF database.

Column descriptions:
* [CMS Codebook pp 4-9](https://www.cms.gov/files/document/de-10-codebook.pdf-0)

Code mappings (sex, race, state and county):
* [CMS Codebook pp 13-18](https://www.cms.gov/files/document/de-10-codebook.pdf-0)

In [2]:
# !pip install tabula-py==2.7.0
# !pip install lxml

In [12]:
PDF_PATH = "https://www.cms.gov/research-statistics-data-and-systems/downloadable-public-use-files/synpufs/downloads/synpuf_dug.pdf"
TABLE_PAGES = {
    "beneficiary_summary": [9,10],
    "inpatient_claims": [11],
    "outpatient_claims": [12],
    "prescription_drug_events": [14],
}

OUTPUT_PATH = '../data/columns_description.csv'

In [4]:
import pandas as pd
import tabula
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [5]:
df_cols = None
columns = ['Variable names', 'Labels']
# df0 = None
for table_name, pages in TABLE_PAGES.items():
    for page in pages:
        cols_desc = tabula.read_pdf(PDF_PATH, output_format='dataframe', silent=True, pages=[page])
        df = cols_desc[0]
        
        unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
        if unnamed_cols:
            unnamed_col = unnamed_cols[0]
            col = df.columns[df.isna().all()].tolist()
            col = col[0]
            df.drop(columns=[col, '#'], inplace=True)
            df.rename(columns={unnamed_col: col}, inplace=True)
        df = df[columns]
        df['table_name'] = table_name
        if df_cols is None:
            df_cols = df.copy()
        else:
            df_cols = pd.concat([df_cols, df], ignore_index=True, axis=0)

columns = {"Variable names": "column_name", "Labels": "column_description"}            
df_cols.rename(columns=columns, inplace=True)

In [6]:
df_cols.head()

column_name               column_description           table_name
0        DESYNPUF_ID       DESYNPUF: Beneficiary Code  beneficiary_summary
1      BENE_BIRTH_DT          DESYNPUF: Date of birth  beneficiary_summary
2      BENE_DEATH_DT          DESYNPUF: Date of death  beneficiary_summary
3  BENE_SEX_IDENT_CD                    DESYNPUF: Sex  beneficiary_summary
4       BENE_RACE_CD  DESYNPUF: Beneficiary Race Code  beneficiary_summary

In [10]:
def remove_suffix(row):
    row['column_description'] = row['column_description'].replace('DESYNPUF:', '').strip()
    return row

In [11]:
df_cols = df_cols.apply(remove_suffix, axis=1)
df_cols.head()

column_name     column_description           table_name
0        DESYNPUF_ID       Beneficiary Code  beneficiary_summary
1      BENE_BIRTH_DT          Date of birth  beneficiary_summary
2      BENE_DEATH_DT          Date of death  beneficiary_summary
3  BENE_SEX_IDENT_CD                    Sex  beneficiary_summary
4       BENE_RACE_CD  Beneficiary Race Code  beneficiary_summary

In [13]:
df_cols.to_csv(OUTPUT_PATH, index=False)